In [1]:
import itertools as it
import pickle as pkl
import os
from collections import OrderedDict
%run ../src/sampling_utils.py
%run ../src/shared_utils.py

In [2]:
disease = "covid19"
prediction_region = "germany"

with open('../data/counties/counties.pkl', "rb") as f:
    counties = pkl.load(f)

data = load_daily_data(disease, prediction_region, counties)
data = data.iloc[:, 0:31]
times = uniform_times_by_day(data.index)
locs = uniform_locations_by_county(counties)

def temporal_bfs(x): return bspline_bfs(x, np.array(
    [0, 0, 1, 2, 3, 4, 5]) * 24 * 3600.0, 2)

def spatial_bfs(x): return [gaussian_bf(x, σ)
                            for σ in [6.25, 12.5, 25.0, 50.0]]

num_features = len(temporal_bfs(tt.fmatrix("tmp"))) * len(spatial_bfs(tt.fmatrix("tmp")))

ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs)

In [3]:
%%timeit
res_new = sample_ia_effects(data, times, locs, ia_bfs, num_features)

/usr/local/software/jureca/Stages/Devel-2019a/software/Python/3.6.8-GCCcore-8.3.0/lib/python3.6/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/usr/local/software/jureca/Stages/Devel-2019a/software/Python/3.6.8-GCCcore-8.3.0/lib/python3.6/site-packages/theano/tensor/subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


3.19 s ± 56.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
samp = IAEffectSampler(data, times, locs, temporal_bfs, spatial_bfs, verbose=False)
res_old = samp(data.index, data.columns)

34.2 s ± 501 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
res_new = sample_ia_effects(data, times, locs, ia_bfs, num_features)
samp = IAEffectSampler(data, times, locs, temporal_bfs, spatial_bfs, verbose=False)
res_old = samp(data.index, data.columns)

np.array_equal(res_old, res_new)

/usr/local/software/jureca/Stages/Devel-2019a/software/Python/3.6.8-GCCcore-8.3.0/lib/python3.6/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/usr/local/software/jureca/Stages/Devel-2019a/software/Python/3.6.8-GCCcore-8.3.0/lib/python3.6/site-packages/theano/tensor/subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


False

In [15]:
res_new[-1,1,:]

array([0.03122144, 0.01563206, 0.01055665, 0.01991831, 0.17515898,
       0.08762576, 0.04986301, 0.06200151, 0.42624635, 0.21315777,
       0.11141692, 0.09576207, 0.5035817 , 0.25180554, 0.12857512,
       0.1088883 ], dtype=float32)

In [16]:
res_old[-1,1,:]

array([0.0100014 , 0.01024917, 0.00885839, 0.0168313 , 0.0373856 ,
       0.04373937, 0.0383384 , 0.05307557, 0.06239312, 0.09210943,
       0.08441518, 0.08601639, 0.08390123, 0.11935139, 0.10128495,
       0.10140499], dtype=float32)